In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# from pyzbar.pyzbar import decode
# from skimage.filters import threshold_local
from sklearn.cluster import DBSCAN

from local_library.image_operations import *


In [ ]:
# def decodeQR(RGB_image):
#     """
#     Decodes a QR code from an RGB image using local thresholding and sigmoid transformation.

#     Parameters:
#     RGB_image (numpy.ndarray): The input image in RGB format.

#     Returns:
#     list: Decoded QR code information, if any is detected.
#     """
#     # Convert RGB image to grayscale using YCbCr luminance calculation
#     ycbcr_constant = np.array([0.299, 0.587, 0.114])
#     y_image = (RGB_image * ycbcr_constant).sum(axis=-1)

#     # Normalize and convert to integer scale if needed
#     y_image = (y_image * 255).astype(int) if y_image.max() <= 1 else y_image.astype(int)

#     # Apply a local threshold to the grayscale image
#     local_thresh = threshold_local(y_image, block_size=65, offset=10)

#     # Perform sigmoid transformation for enhanced contrast
#     sigmoid_local = (1 / (1 + np.exp(-10 * ((y_image / 255) - (local_thresh / 255)))) * 255).astype(int)

#     # Decode the QR code from the processed image
#     decode_sigmoid_local = decode(sigmoid_local)

#     return decode_sigmoid_local


In [ ]:
# Directory for seedling images
FOLDER_DIRECTION = 'D:/#Work/4-1 WiL/Workspace/Dataset/seedling_pi_webcam/EcRfSd_03_4/'

# Thresholds for mean Y-value in image selection for different processes
QR_DETECTION_IMAGE_Y_MEAN_THRESHOLD  = 0.05
SEEDLING_STATUS_UPDATE_IMAGE_Y_MEAN_THRESHOLD  = 0.3

# Blue mark color characteristics in YCbCr space
BLUE_MARK_COLOR_MEAN  = np.array([0.11555396, -0.06690868])
BLUE_MARK_COLOR_COVARIANCE  = np.array([[8.34169839e-04, -7.15137086e-05],
                                        [-7.15137086e-05,  1.78775272e-04]])

# Maximum pixel count and cluster count for blue mark detection before skipping
BLUE_MARK_MAX_PIXEL_THRESHOLD  = 800
BLUE_MARK_MAX_CLUSTER_THRESHOLD  = 10
# Threshold for determining blue mark probability
BLUE_MARK_PROBABILITY_THRESHOLD  = 0.1

# Reference shape distances and shape ratio for mark detection
MARK_REFERENCE_SHAPE_DISTANCES  = np.array([ 29, 54.5, (54.5 ** 2 + 29 ** 2) ** 0.5])
# min and max ratio thresholds for acceptable shape
MIN_MARK_SHAPE_RATIO_THRESHOLD  = ((MARK_REFERENCE_SHAPE_DISTANCES  - 2) / MARK_REFERENCE_SHAPE_DISTANCES.min()).repeat(2)
MAX_MARK_SHAPE_RATIO_THRESHOLD  = ((MARK_REFERENCE_SHAPE_DISTANCES  + 2) / MARK_REFERENCE_SHAPE_DISTANCES.min()).repeat(2)


In [ ]:

# Get and sort list of image names
image_name_list = os.listdir(FOLDER_DIRECTION)
image_name_list.sort()

# Dictionary to store grouped images by timestamp
grouped_images = {}

# Group images by timestamp
for image_name in image_name_list:
    timestamp = '_'.join(image_name.split('_')[3:5])  # Extract timestamp (date and hour)
    if timestamp not in grouped_images:
        grouped_images[timestamp] = []
    grouped_images[timestamp].append(image_name)

# Dictionary to store the filtered images by hour
filtered_data = {}

# Iterate over grouped images to filter by hour
for timestamp, images in grouped_images.items():
    # Extract date and hour (YYYY-MM-DD_HH)
    date_hour = timestamp[:13]

    # Check if this date_hour is already in filtered_data
    if date_hour in filtered_data:
        # Compare by number of images and by timestamp to keep the appropriate key
        current_timestamp, current_images = filtered_data[date_hour]
        if len(images) > len(current_images) or (len(images) == len(current_images) and timestamp < current_timestamp):
            filtered_data[date_hour] = (timestamp, images)
    else:
        # Store the first occurrence of this date_hour
        filtered_data[date_hour] = (timestamp, images)

# Rebuild the final dictionary with filtered results
grouped_images = {key: value for key, value in filtered_data.values()}

print(grouped_images)

In [ ]:
timestamp_list = list(grouped_images.keys())
num_timestamp = len(timestamp_list)
update_seedling_status_image_name_list, qr_detection_image_name_list, blue_mark_detection_image_name_list = [], [], []
for timestamp in timestamp_list:
    num_image_per_timestamp = len(grouped_images[timestamp])
    y_mean_buffer = np.zeros(num_image_per_timestamp)
    for image_index in range(num_image_per_timestamp):
        rgb_image = plt.imread(f'{FOLDER_DIRECTION}{grouped_images[timestamp][image_index]}')[:,:,:3]
        image_shape = rgb_image.shape
        y_mean_buffer[image_index] = (rgb_image * np.array([0.299, 0.587, 0.114])).sum(axis=-1)[:,image_shape[1]//3:-image_shape[1]//3].mean()
    update_seedling_status_image_index = np.abs(y_mean_buffer - SEEDLING_STATUS_UPDATE_IMAGE_Y_MEAN_THRESHOLD ).argmin()
    update_seedling_status_image_name_list.append(grouped_images[timestamp][update_seedling_status_image_index])
    qr_detection_image_index = np.abs(y_mean_buffer - QR_DETECTION_IMAGE_Y_MEAN_THRESHOLD).argmin()
    qr_detection_image_name_list.append(grouped_images[timestamp][qr_detection_image_index])


In [ ]:
# for i in range(num_timestamp):    
#     rgb_img = plt.imread(f'{FOLDER_DIRECTION}{qr_detection_image_name_list[i]}')
#     print(f'{qr_detection_image_name_list[i]} : {len(decodeQR(rgb_img))}')

In [ ]:
# plt.figure(figsize=(25, 150))
# for i in range(num_timestamp):
#     plt.subplot(30,3,i+1), plt.imshow(plt.imread(f'{FOLDER_DIRECTION}{update_seedling_status_image_name_list[i]}'))

In [ ]:
BLUE_MARK_MAX_PIXEL_THRESHOLD  = 1500
BLUE_MARK_MAX_CLUSTER_THRESHOLD  = 20

In [ ]:
len(blue_mark_mask_position)

In [ ]:
number_of_mark

In [ ]:
tray_corner_position = np.zeros((4,2))
tray_orientation = 'RIGHT'
plt.figure(figsize=(25, 150))
for image_index in range(num_timestamp):
# for image_index in range(3):  
    
    rgb_img = plt.imread(f'{FOLDER_DIRECTION}{update_seedling_status_image_name_list[image_index]}')[:,:,:3]
        
    ycbcr_img = convertRGBtoYCbCr(rgb_img)
    
    ycbcr_mean = ycbcr_img.reshape(-1,3).mean(0)
    blue_mark_color_mean_offset = np.array([
        (ycbcr_mean[1] - 0.006) * 2 if ycbcr_mean[1] > 0.006 else 0,
        (ycbcr_mean[2] - 0.002) * 0.05 if ycbcr_mean[2] < -0.002 else 0
    ])

    blue_mark_probability = calculateMultiGaussianDist(ycbcr_img[:,:,1:], BLUE_MARK_COLOR_MEAN  + blue_mark_color_mean_offset, BLUE_MARK_COLOR_COVARIANCE )
    blue_mark_probability /= blue_mark_probability.max()
    
    blue_mark_mask_position = np.stack(np.where(blue_mark_probability > BLUE_MARK_PROBABILITY_THRESHOLD ), axis = -1)
        
        
    plt.subplot(30,3,image_index+1)
    found = False

    
    if len(blue_mark_mask_position) < BLUE_MARK_MAX_PIXEL_THRESHOLD  :
        labels = DBSCAN(eps = 5, min_samples = 5).fit_predict(blue_mark_mask_position)
        # Calculate the number of distinct seedlings
        number_of_mark = labels.max() + 1
        
        if number_of_mark < BLUE_MARK_MAX_CLUSTER_THRESHOLD  :
            blue_mark_position = np.array([
                blue_mark_mask_position[labels == index].mean(0) for index in range(number_of_mark)
            ])
            
            # Calculate the distance matrix between points
            mark_distance_matrix = calculateEuclidean(blue_mark_position[:, np.newaxis, :], blue_mark_position[np.newaxis, :, :])
            found = False

            # Loop through unique combinations of four points
            for point_a_index in range(len(blue_mark_position) - 3):
                if found: break
                for point_b_index in range(point_a_index + 1, len(blue_mark_position) - 2):
                    if found: break
                    for point_c_index in range(point_b_index + 1, len(blue_mark_position) - 1):
                        if found: break
                        for point_d_index in range(point_c_index + 1, len(blue_mark_position)):
                            # Select indices and calculate distances
                            index = np.array([point_a_index, point_b_index, point_c_index, point_d_index])
                            selected_distances = mark_distance_matrix[np.ix_(index, index)]
                            sorted_distances = np.sort(selected_distances.reshape(-1))[4::2]
                            # avg_distances = sorted_distances.reshape(3, 2).mean(axis=1)
                            
                            # Calculate shape ratio and check within range
                            sorted_distances_norm = sorted_distances / sorted_distances.min()
                            if np.all((MIN_MARK_SHAPE_RATIO_THRESHOLD  < sorted_distances_norm) & (sorted_distances_norm < MAX_MARK_SHAPE_RATIO_THRESHOLD )):
                                
                                # Determine corner quadrants
                                relative_positions = (blue_mark_position[index] - blue_mark_position[index].mean(0)).clip(-1, 1).astype(int)
                                corner_quadrant_ref_order = np.array([[-1, -1], [-1, 1], [1, 1], [1, -1]])
                                
                                # Adjust based on tray orientation
                                rotation_shift = {'UP': 1, 'LEFT': 2, 'DOWN': 3}.get(tray_orientation, 0)
                                corner_quadrant_ref_order = np.roll(corner_quadrant_ref_order, shift=rotation_shift, axis=0)
                                
                                # Map quadrants to order and extract corner positions
                                order = np.argmax((corner_quadrant_ref_order[:, None] == relative_positions).all(-1), axis=0)
                                
                                # tray_corner_position_l = tray_corner_position
                                
                                tray_corner_position = blue_mark_position[index][order]
                                
                                # print(f'{update_seedling_status_image_name_list[image_index]} : {(tray_corner_position_l - tray_corner_position).mean()}')
                                
                                found = True
                                break

            
            

            # mask = np.where(like > like_thresh, 1, 0).astype('uint8')
            # plt.subplot(30,3,i+1), plt.imshow(mask)
            # plt.subplot(30,3,i+1), plt.scatter(mask_position[:, 1], mask_position[:, 0], c=labels, cmap='jet', s=0.5), plt.xlim([0, 1920]), plt.ylim([0, 1080]), plt.colorbar()
            # plt.subplot(30,3,image_index+1)
            plt.scatter(blue_mark_position[:, 1], blue_mark_position[:, 0], c=np.arange(number_of_mark), cmap='jet', s=10), plt.xlim([0, 1920]), plt.ylim([0, 1080]), plt.colorbar() 
        else:
            plt.scatter(blue_mark_position[:, 1], blue_mark_position[:, 0], s=10), plt.xlim([0, 1920]), plt.ylim([0, 1080]) 
    else:
        plt.scatter(blue_mark_position[:, 1], blue_mark_position[:, 0], s=10), plt.xlim([0, 1920]), plt.ylim([0, 1080])
            # plt.title(f'{update_seedling_status_image_name_list[image_index]} \nnumber_of_mark:{number_of_mark}, mask:{len(blue_mark_mask_position)}, found:{found}')
    plt.scatter(tray_corner_position[:, 1], tray_corner_position[:, 0], s=90, alpha=1, facecolors='none', edgecolors='black' ), plt.xlim([0, 1920]), plt.ylim([0, 1080])
    plt.title(f'{update_seedling_status_image_name_list[image_index]} \nfound:{found}')
            
            

In [ ]:
plt.figure(figsize=(25, 300))
for image_index in range(num_timestamp):
# for image_index in range(2):  
    
    qr_detection_img = plt.imread(f'{FOLDER_DIRECTION}{qr_detection_image_name_list[image_index]}')[:,:,:3]
    rgb_img = plt.imread(f'{FOLDER_DIRECTION}{update_seedling_status_image_name_list[image_index]}')[:,:,:3]
    
    qr_detection_result = decodeQR(qr_detection_img)
    if len(qr_detection_result) > 0:
        tray_id = qr_detection_result[0].data.decode("utf-8").split('.')[0]
        tray_orientation = qr_detection_result[0].orientation
    
    ycbcr_img = convertRGBtoYCbCr(rgb_img)
    ycbcr_mean = ycbcr_img.reshape(-1,3).mean(0)
    blue_mark_color_mean_offset = np.array([
        (ycbcr_mean[1] - 0.006) * 2 if ycbcr_mean[1] > 0.006 else 0,
        (ycbcr_mean[2] - 0.002) * 0.05 if ycbcr_mean[2] < -0.002 else 0
    ])

    blue_mark_probability = calculateMultiGaussianDist(ycbcr_img[:,:,1:], BLUE_MARK_COLOR_MEAN  + blue_mark_color_mean_offset, BLUE_MARK_COLOR_COVARIANCE )
    blue_mark_probability /= blue_mark_probability.max()
    
    blue_mark_mask_position = np.stack(np.where(blue_mark_probability > BLUE_MARK_PROBABILITY_THRESHOLD ), axis = -1)
        
        
    corner_tray_found = False
    
    if len(blue_mark_mask_position) < BLUE_MARK_MAX_PIXEL_THRESHOLD  :
        labels = DBSCAN(eps = 5, min_samples = 5).fit_predict(blue_mark_mask_position)
        # Calculate the number of distinct seedlings
        number_of_mark = labels.max() + 1
        
        if number_of_mark < BLUE_MARK_MAX_CLUSTER_THRESHOLD  :
            blue_mark_position = np.array([
                blue_mark_mask_position[labels == index].mean(0) for index in range(number_of_mark)
            ])
            
            # Calculate the distance matrix between points
            mark_distance_matrix = calculateEuclidean(blue_mark_position[:, np.newaxis, :], blue_mark_position[np.newaxis, :, :])
            corner_tray_found = False

            # Loop through unique combinations of four points
            for point_a_index in range(len(blue_mark_position) - 3):
                if corner_tray_found: break
                for point_b_index in range(point_a_index + 1, len(blue_mark_position) - 2):
                    if corner_tray_found: break
                    for point_c_index in range(point_b_index + 1, len(blue_mark_position) - 1):
                        if corner_tray_found: break
                        for point_d_index in range(point_c_index + 1, len(blue_mark_position)):
                            # Select indices and calculate distances
                            index = np.array([point_a_index, point_b_index, point_c_index, point_d_index])
                            selected_distances = mark_distance_matrix[np.ix_(index, index)]
                            sorted_distances = np.sort(selected_distances.reshape(-1))[4::2]
                            # avg_distances = sorted_distances.reshape(3, 2).mean(axis=1)
                            
                            # Calculate shape ratio and check within range
                            sorted_distances_norm = sorted_distances / sorted_distances.min()
                            if np.all((MIN_MARK_SHAPE_RATIO_THRESHOLD  < sorted_distances_norm) & (sorted_distances_norm < MAX_MARK_SHAPE_RATIO_THRESHOLD )):
                                
                                # Determine corner quadrants
                                relative_positions = (blue_mark_position[index] - blue_mark_position[index].mean(0)).clip(-1, 1).astype(int)
                                corner_quadrant_ref_order = np.array([[-1, -1], [-1, 1], [1, 1], [1, -1]])
                                
                                # Adjust based on tray orientation
                                rotation_shift = {'UP': 1, 'LEFT': 2, 'DOWN': 3}.get(tray_orientation, 0)
                                corner_quadrant_ref_order = np.roll(corner_quadrant_ref_order, shift=rotation_shift, axis=0)
                                
                                # Map quadrants to order and extract corner positions
                                order = np.argmax((corner_quadrant_ref_order[:, None] == relative_positions).all(-1), axis=0)
                                
                                # tray_corner_position_l = tray_corner_position
                                
                                tray_corner_position = blue_mark_position[index][order]
                                
                                # print(f'{update_seedling_status_image_name_list[image_index]} : {(tray_corner_position_l - tray_corner_position).mean()}')
                                
                                corner_tray_found = True
                                break
    if corner_tray_found == False and image_index == 0:
        print(f'first tray cannot detect blue mark')
        break
    

        
    plt.subplot(50,3,image_index+1)
    rgb_image_roi = warpPerspectiveTransformation(rgb_img, tray_corner_position[:,::-1], 290, 545)
    plt.subplot(50,3,image_index+1), plt.imshow(rgb_image_roi)
    plt.title(f'{update_seedling_status_image_name_list[image_index]}')
            
            